Back to Instance Properties

In [1]:
class IntegerValue:

    def __init__(self):
        self.values = {} 

    def __set__(self, instance, value):
        self.values[instance] = int(value) 

    def __get__(self, instance, owner_class):
        return self if instance is None else self.values.get(instance)
    


`self.values[instance] = value `

Store a instance as a dictionary key is problematic bc the strong reference creation. Even if the instance is deleted, the reference count never reaches zero. We can use Weak references to solve the problem:

In [2]:
import weakref

In [7]:
class IntegerValue:

    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value) 

    def __get__(self, instance, owner_class):
        return self if instance is None else self.values.get(instance)
    


Assuming that the isntance is hashable, that should work

In [8]:
class Point:
    x = IntegerValue()

p = Point()

In [9]:
print(hex(id(p)))

0x1f3afa88a60


In [10]:
p.x = 100.1 
p.x

100

In [11]:
Point.x.values.keyrefs()

[<weakref at 0x000001F3AF232750; to 'Point' at 0x000001F3AFA88A60>]

If i delete `p`, then i'm deleting the only strong reference

In [12]:
del p

In [13]:
Point.x.values.keyrefs()

[]

If we __don't have the hashable object__, we can't use them as keys in a dictionary, not even a weak one

We can use the id of the instance as a hash key (not the instance directly)

In [14]:
class IntegerValue:

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value) 

    def __get__(self, instance, owner_class):
        return self if instance is None else self.values.get(id(instance))
    


In [17]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x 

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x 

p = Point(10)

In [18]:
p.x

10

In [20]:
p.x = 20 
p.x

20

In [21]:
id(p), Point.x.values

(2146131353392, {2146131353392: 20})

In [22]:
import ctypes

def ref_count(id):
    return ctypes.c_long.from_address(id).value

In [24]:
p_id = id(p)
ref_count(p_id)

1

In [27]:
del p 

NameError: name 'p' is not defined

In [28]:
ref_count(p_id)

0

The strong references count is zero, but the key in the dictionary is still there

In [29]:
Point.x.values

{2146131353392: 20}

Python also reuse memory addresses, and those id's can be reused.

We need a way __to discover when the object has ben removed and remove also the key in that dictionary__

Normally, we use weak references bc they're aware when the object is destroyed, but that solution only work to hashable objects 

In [30]:
p = Point(10.1)

weak_p = weakref.ref(p)

In [31]:
weak_p

<weakref at 0x000001F3AF70B830; to 'Point' at 0x000001F3AF8F3970>

In [32]:
ref_count(id(p))

1

In [33]:
del p 

In [34]:
weak_p 

<weakref at 0x000001F3AF70B830; dead>

When we're creating weak references, we can pass __as a parameter a callback function__, a function that is called when the object is dead

In [35]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [36]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [37]:
del p 

<weakref at 0x000001F3AFD527A0; dead> is being destroyed


We're not only storing the value as the value, we're storing the value as a weak reference to the object and the value that we want associated to it

In [39]:
class IntegerValue:

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
        
    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')

In [40]:
class Point:
    x = IntegerValue()

p1 = Point()
p2 = Point()

p1.x, p2.x = 10, 20

In [41]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [42]:
del p1

removing dead entry for <weakref at 0x000001F3AFCB51C0; dead>


In [43]:
class IntegerValue:

    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
        
    def _remove_object(self, weak_ref):
        # reverse_lookup = [key for key, value in self.values.items()
        #                   if value[0] is weak_ref]
        
        # if reverse_lookup:
        #     key = reverse_lookup[0]
        #     del self.values[key]

        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break 

In [44]:
class Point:
    x = IntegerValue()

p = Point()

In [45]:
p.x = 10.1 

p.x

10

In [46]:
Point.x.values

{2146135653824: (<weakref at 0x000001F3AF790B30; to 'Point' at 0x000001F3AFA73DC0>,
  10)}

In [47]:
del p 

In [48]:
Point.x.values

{}

Another problem now: when we create weak references, the weak reference is stored in a key in the class dictionary

In [49]:
class Person:
    pass 

Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [50]:
hasattr(Person.__weakref__, '__get__')

True

In [51]:
p = Person
hasattr(p, '__weakref__')

True

In [52]:
print(p.__weakref__)

<attribute '__weakref__' of 'Person' objects>


In [53]:
w = weakref.ref(p)

print(p.__weakref__)

<attribute '__weakref__' of 'Person' objects>


We see only one item, but can have more than one

In [54]:
class Person:
    __slots__ = 'name', 

In [55]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [56]:
p = Person()

In [57]:
hasattr(p, '__weakref__')

False

The problem is that __when we use the slots, we canno't have weak references to this object anymore__ because it doesn't have the `__weakref__` attribute anymore

In [58]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In order to solve that, we need to have `__weakref__` in the slots parameter

In [59]:
class Person:
    __slots__ = 'name', '__weakref__'

In [61]:
p = Person()
weakref.ref(p)

<weakref at 0x000001F3AFE60950; to 'Person' at 0x000001F3AFA2C5B0>

## Another example

In [80]:
class ValidString:
    def __init__(self, min_len=0, max_len=255):
        self.data = {}
        self._min_len = min_len
        self._max_len = max_len

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string')
        
        if len(value) < self._min_len:
            raise ValueError(f'Value muxt be at least {self._min_len} characters')
        
        if len(value) > self._max_len:
            raise ValueError(f'Value must not transpass {self._max_len} characters long')
        
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
        
    def _finalize_instance(self, weak_ref):
        for key, value in self.data.items():
            if value[0] is weak_ref:
                del self.data[key]
                break 


In [81]:
class Person:
    __slots__ = '__weakref__',

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return (
            isinstance(other, Person) and 
            self.first_name == other.first_name and             
            self.last_name == other.last_name
        )

By creating __eq__ method, Person class is not hashable anymore

In [82]:
class BankAccount:
    __slots__ = '__weakref__', 

    account_number = ValidString(5, 255)

    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [83]:
p1 = Person()

In [84]:
p1.first_name = ''

ValueError: Value muxt be at least 1 characters

In [85]:
p2 = Person()

In [86]:
p1.first_name, p1.last_name = 'Lorena', 'Miranda'
p2.first_name, p2.last_name = 'Carol', 'Coelho'


In [87]:
b1, b2 = BankAccount(), BankAccount()

In [92]:
b1.account_number = '123456'
b2.account_number = '789101'


In [89]:
p1.first_name, p1.last_name

('Lorena', 'Miranda')

In [93]:
b1.account_number, b2.account_number

('123456', '789101')

In [95]:
Person.first_name.data

{2146135915056: (<weakref at 0x000001F3AFFD88B0; to 'Person' at 0x000001F3AFAB3A30>,
  'Lorena'),
 2146136400816: (<weakref at 0x000001F3AFFDAD40; to 'Person' at 0x000001F3AFB2A3B0>,
  'Carol')}

In [96]:
Person.last_name.data

{2146135915056: (<weakref at 0x000001F3AF689A30; to 'Person' at 0x000001F3AFAB3A30>,
  'Miranda'),
 2146136400816: (<weakref at 0x000001F3AF74C220; to 'Person' at 0x000001F3AFB2A3B0>,
  'Coelho')}

In [97]:
BankAccount.account_number.data

{2146134143760: (<weakref at 0x000001F3B1018B80; to 'BankAccount' at 0x000001F3AF903310>,
  '123456'),
 2146134141648: (<weakref at 0x000001F3AFFE18F0; to 'BankAccount' at 0x000001F3AF902AD0>,
  '789101')}

In [98]:
del p1
del p2
del b1
del b2

In [99]:
Person.first_name.data

{2146135915056: (<weakref at 0x000001F3AFFD88B0; to 'Person' at 0x000001F3AFAB3A30>,
  'Lorena')}

Left over reference... something is not zero and this is a __jupyter__ problem. When we had the value error exception, another reference is created. To avoid that, we need to have the error catched by a try/except block